<a href="https://colab.research.google.com/github/Elieser1945/DeepLearning/blob/main/Week_2_Classification_Coverdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Model MLP Dengan PyTorch dan TensorFlow**

# 1. Model PyTorch

#import libraries

In [2]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load Dataset

In [4]:
df = pd.read_csv("/content/compressed_data.csv.gz")

# Membuat Dataset untuk Regresi

In [6]:
# Hapus baris dengan NaN di target
df = df.dropna(subset=['Cover_Type'])


# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"] - 1  # Mengurangi 1 agar kelas mulai dari 0 (karena PyTorch membutuhkan label 0-based)


# Bagi dataset menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Konversi ke Tensor PyTorch
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
y_test_torch = torch.tensor(y_test.values, dtype=torch.long)

# Membangun Model MLP

In [7]:

# Definisi Model PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Inisialisasi model
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(np.unique(y))
model_torch = MLPModel(input_size, hidden_size, num_classes)


# Definisi loss function dan optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_torch.parameters(), lr=0.001)


# Early stopping manual PyTorch
early_stop_patience = 5
best_loss = float("inf")
patience = 0

# Training model PyTorch

In [ ]:

for epoch in range(200):
    optimizer.zero_grad()
    outputs = model_torch(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

    # Validasi
    with torch.no_grad():
        val_outputs = model_torch(X_test_torch)
        val_loss = criterion(val_outputs, y_test_torch)

    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Early stopping
    if val_loss.item() < best_loss:
        best_loss = val_loss.item()
        patience = 0
    else:
        patience += 1
    if patience >= early_stop_patience:
        print("Early stopping!")
        break

Epoch 1, Loss: 1.8829834461212158, Val Loss: 1.8428910970687866
Epoch 2, Loss: 1.8428775072097778, Val Loss: 1.8040293455123901
Epoch 3, Loss: 1.8039824962615967, Val Loss: 1.7663599252700806
Epoch 4, Loss: 1.7662723064422607, Val Loss: 1.7298469543457031
Epoch 5, Loss: 1.7297084331512451, Val Loss: 1.6944513320922852
Epoch 6, Loss: 1.694251537322998, Val Loss: 1.660136103630066
Epoch 7, Loss: 1.6598676443099976, Val Loss: 1.6268675327301025
Epoch 8, Loss: 1.6265250444412231, Val Loss: 1.5946124792099
Epoch 9, Loss: 1.594191074371338, Val Loss: 1.5633388757705688
Epoch 10, Loss: 1.5628360509872437, Val Loss: 1.5330162048339844
Epoch 11, Loss: 1.532429814338684, Val Loss: 1.5036154985427856
Epoch 12, Loss: 1.5029443502426147, Val Loss: 1.4751086235046387
Epoch 13, Loss: 1.4743516445159912, Val Loss: 1.4474668502807617
Epoch 14, Loss: 1.4466248750686646, Val Loss: 1.4206658601760864
Epoch 15, Loss: 1.419738531112671, Val Loss: 1.3946832418441772
Epoch 16, Loss: 1.3936718702316284, Val Lo

# Evaluasi Model PyTorch

In [ ]:

y_pred_torch = torch.argmax(model_torch(X_test_torch), axis=1).numpy()
print("Akurasi:", accuracy_score(y_test, y_pred_torch))
print("Presisi:", precision_score(y_test, y_pred_torch, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_torch, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_torch, average='weighted'))

Akurasi: 0.7291636188394448
Presisi: 0.7247375533871977
Recall: 0.7291636188394448
F1 Score: 0.7134739203324851


# 2. Model TensorFlow

In [ ]:
# Konversi ke TensorFlow
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.int32)


# Definisi Model TensorFlow
model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Kompilasi model
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Early stopping TensorFlow
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Training model TensorFlow
model_tf.fit(X_train_tf, y_train_tf, epochs=30, validation_data=(X_test_tf, y_test_tf), callbacks=[early_stopping])

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14526/14526 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - accuracy: 0.7362 - loss: 0.6261 - val_accuracy: 0.7843 - val_loss: 0.4994
Epoch 2/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step - accuracy: 0.7884 - loss: 0.4906 - val_accuracy: 0.8020 - val_loss: 0.4672
Epoch 3/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - accuracy: 0.8055 - loss: 0.4565 - val_accuracy: 0.8125 - val_loss: 0.4459
Epoch 4/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8166 - loss: 0.4348 - val_accuracy: 0.8281 - val_loss: 0.4194
Epoch 5/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.8232 - loss: 0.4207 - val_accuracy: 0.8276 - val_loss: 0.4163
Epoch 6/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - accuracy: 0.8288 - loss: 0.4089 - val_accuracy: 0.8316 - val_loss: 0.4015
Epoch 7/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.8326 - loss: 0.4010 - val_accuracy: 0.8373 - val_loss: 0.3903
Epoch 8/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8372 - lo

# Evaluasi Hasil TensorFlow

In [ ]:

y_pred_tf = np.argmax(model_tf.predict(X_test_tf), axis=1)
print("\nEvaluasi TensorFlow:")
print("Akurasi:", accuracy_score(y_test, y_pred_tf))
print("Presisi:", precision_score(y_test, y_pred_tf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_tf, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_tf, average='weighted'))

try:
    print("ROC AUC Score:", roc_auc_score(y_test, model_tf.predict(X_test_tf), multi_class='ovr'))
except ValueError as e:
    print(f"ROC AUC Score tidak dapat dihitung: {e}")


3632/3632 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step

Evaluasi TensorFlow:
Akurasi: 0.8618107966231509
Presisi: 0.8621953112289097
Recall: 0.8618107966231509
F1 Score: 0.8618037765055826
3632/3632 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
ROC AUC Score: 0.9838398807623617
